In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

doc_path = "dev-data/Be_Good.pdf"
loader = PyPDFLoader(doc_path)

doc = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
doc_splits = text_splitter.split_documents(doc)

chromadb = Chroma.from_documents(
    documents=doc_splits,
    embedding=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
)
retriever = chromadb.as_retriever()

In [14]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """You are a helpful assistant that generates multiple sub-questions related to an input 
question.\n The goal is to break down the input into a set of sub-problems / sub-questions that can be 
answers in isolation. (Directly return the queries, don't add a phrase before the queries)\n Generate multiple search queries related to : {question}\n Ouput (3 queries):
"""

prompt_decompositions = ChatPromptTemplate.from_template(template)
llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash-preview-05-20", temperature=0)

sub_question_generator_chain = (
    prompt_decompositions 
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

question = "What is the name of the writter of the book and what the book says about ?"
questions = sub_question_generator_chain.invoke({"question": question})

In [15]:
questions

['[Book Name] author', '[Book Name] summary', '[Book Name] main themes']

In [16]:
from langchain import hub
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
    """RAG on each sub-question"""
    sub_questions = sub_question_generator_chain.invoke({"question": question})
    rag_results = []

    for sub_question in sub_questions:
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs,
                                                               "question": sub_question})
        rag_results.append(answer)

    return rag_results, sub_questions

C:\Users\Tommy RAMAROKOTO\Documents\programmation\projects\New folder\chatbots\backend\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
answers, questions = retrieve_and_rag(question, prompt_rag, sub_question_generator_chain)

C:\Users\Tommy RAMAROKOTO\AppData\Local\Temp\ipykernel_2772\3895697035.py:15: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(sub_question)


In [18]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

'The writer is Paul Graham. The work, titled "Be Good - Essay by Paul Graham," discusses concepts like benevolence and "being good" as a strategy. Its main themes revolve around the inherent value and advantages of benevolence, suggesting that doing good provides a sense of mission, encourages others to offer help, and that avoiding evil can lead to long-term success.'